# The Battle of Neighborhoods - Final

## Download and Explore Dataset

In [1]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import geocoder
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests

#### Load and explore the data

In [2]:
# Postal codes of Nova Scotia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_B'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.body.table.tbody

In [3]:
cells = table.find_all('td')
postcode = []
neighborhood = []

for cell in cells:
    temp = cell.text.strip()
    postcode.append(temp[:3])
    neighborhood.append(temp[3:])

In [4]:
ns_data = pd.DataFrame(data=[postcode, neighborhood]).T
ns_data.columns = ['Postcode', 'Neighborhood']
ns_data = ns_data[ns_data['Neighborhood'] != 'Not assigned'].reset_index(drop=True)
ns_data.head()

,Postcode,Neighborhood
0,B1A,Glace Bay
1,B2A,North SydneySouth Central
2,B3A,DartmouthNorth
3,B4A,BedfordSoutheast
4,B5A,Yarmouth


In [5]:
ns_data.shape

(63, 2)

Data from https://www.serviceobjects.com/blog/free-zip-code-and-postal-code-database-with-geocoordinates/

In [6]:
f = pd.read_csv('CanadianPostalCodes202101.csv')
f['POSTAL_CODE'] = f['POSTAL_CODE'].map(lambda x: x[:3])

df = pd.DataFrame(f[['POSTAL_CODE', 'LATITUDE', 'LONGITUDE']]).rename(columns={
    'POSTAL_CODE':'Postcode', 'LATITUDE':'Latitude', 'LONGITUDE':'Longitude'})
df.head()

,Postcode,Latitude,Longitude
0,A0A,47.058805,-52.890816
1,A0A,47.571958,-53.278126
2,A0A,46.836490,-52.956880
3,A0A,47.739416,-53.229699
4,A0B,47.668083,-53.826107


In [7]:
geodata = df.groupby(['Postcode']).median()

ns_data = ns_data.merge(geodata, how='left', on='Postcode').dropna().reset_index(drop=True)
ns_data.head()

,Postcode,Neighborhood,Latitude,Longitude
0,B1A,Glace Bay,46.192602,-59.961457
1,B2A,North SydneySouth Central,46.206119,-60.258852
2,B3A,DartmouthNorth,44.678562,-63.579393
3,B4A,BedfordSoutheast,44.726004,-63.668742
4,B5A,Yarmouth,43.838441,-66.114026


In [8]:
ns_data.shape

(63, 4)

In [9]:
def getMap(address, dataset, zoom_start=10):
    geolocator = Nominatim(user_agent=f'{address}_explorer')
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print(f'The geograpical coordinate of {address} are {latitude}, {longitude}.')
    
    # create map of given address using latitude and longitude values
    Map = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)

    # add markers to map
    for lat, lng, neighborhood in zip(dataset['Latitude'], dataset['Longitude'], dataset['Neighborhood']):
        label = f'{neighborhood}'
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(Map)
        
    return Map

In [10]:
getMap('Halifax', ns_data, 7)

The geograpical coordinate of Halifax are 44.648618, -63.5859487.


![](image/ns.png)

We will only use the HRM(Halifax Regional Municipal) area.

In [11]:
hrm_data = ns_data.sort_values(by=['Postcode']).reset_index(drop=True)[28:55].reset_index(drop=True)
hrm_data.head()

,Postcode,Neighborhood,Latitude,Longitude
0,B2T,Enfield / Fall River(YHZ),44.858733,-63.591856
1,B2V,Dartmouth(Morris Lake / Cole Harbour),44.662519,-63.483692
2,B2W,DartmouthEast Central(Portland Estates / South...,44.678888,-63.510144
3,B2X,DartmouthNorth Central,44.697761,-63.541843
4,B2Y,DartmouthSouth Central(North Woodside),44.667825,-63.552783


In [12]:
getMap('Halifax', hrm_data, 10)

The geograpical coordinate of Halifax are 44.648618, -63.5859487.


![](image/hrm.png)

In [13]:
# lattitude and longitude data for Seoul and its metropolitian area
seoul_data = pd.read_csv('Seoul.csv')
seoul_data.head()

,Neighborhood,Latitude,Longitude
0,Seoul/Jongno District,37.592128,126.979420
1,Seoul/Jung District,37.557335,126.997985
2,Seoul/Yongsan District,37.528582,126.981987
3,Seoul/Seongdong District,37.548240,127.043114
4,Seoul/Gwangjin District,37.543059,127.088351


In [14]:
getMap('Seoul', seoul_data, 9)

The geograpical coordinate of Seoul are 37.5666791, 126.9782914.


![](image/seoul.png)

In [15]:
hrm_data = hrm_data.drop(columns=['Postcode'])
hrm_data['Region'] = 'HRM'
seoul_data['Region'] = 'Seoul'


hrm_seoul = pd.concat([hrm_data, seoul_data]).reset_index(drop=True)
hrm_seoul_cols = [hrm_seoul.columns[-1]]+list(hrm_seoul.columns[:-1])
hrm_seoul = hrm_seoul[hrm_seoul_cols]
hrm_seoul.head()

,Region,Neighborhood,Latitude,Longitude
0,HRM,Enfield / Fall River(YHZ),44.858733,-63.591856
1,HRM,Dartmouth(Morris Lake / Cole Harbour),44.662519,-63.483692
2,HRM,DartmouthEast Central(Portland Estates / South...,44.678888,-63.510144
3,HRM,DartmouthNorth Central,44.697761,-63.541843
4,HRM,DartmouthSouth Central(North Woodside),44.667825,-63.552783


In [16]:
hrm_seoul.shape

(72, 4)

In [17]:
CLIENT_ID = 'DXPKHFEPVLSRGBWMN4Y3AP11X0I00DITNPNVJUDJFVGGS34N' # your Foursquare ID
CLIENT_SECRET = 'JXKX4OHGPSKEZUK0SFA0NHXLUMJDGCNHPZLOFEKW3MONV2BU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DXPKHFEPVLSRGBWMN4Y3AP11X0I00DITNPNVJUDJFVGGS34N
CLIENT_SECRET:JXKX4OHGPSKEZUK0SFA0NHXLUMJDGCNHPZLOFEKW3MONV2BU


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={LIMIT}'
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
hrm_seoul_venues = getNearbyVenues(names=hrm_seoul['Neighborhood'],
                                   latitudes=hrm_seoul['Latitude'],
                                   longitudes=hrm_seoul['Longitude']
                                  )

Enfield / Fall River(YHZ)
Dartmouth(Morris Lake / Cole Harbour)
DartmouthEast Central(Portland Estates / South Woodside / Woodlawn)
DartmouthNorth Central
DartmouthSouth Central(North Woodside)
DartmouthEast(East Lawrencetown / Preston / Mineville / Upper Lawrencetown)
DartmouthNorth
DartmouthNorthwest(Burnside)
Porters Lake
Eastern Passage
HalifaxLower Harbour
HalifaxMid-Harbour

Nova Scotia Provincial Government
HalifaxUpper Harbour

Canadian Forces(MARLANT)
HalifaxCentral
HalifaxBedford Basin
HalifaxSouth Central
HalifaxNorth West Arm
HalifaxSouth
HalifaxWest(Bayers Lake / Clayton Park)
Timberlea
Williamswood
Upper Tantallon
BedfordSoutheast
BedfordNorthwest
Lower SackvilleSouth
Lower SackvilleWest
Lower SackvilleNorth
Seoul/Jongno District
Seoul/Jung District
Seoul/Yongsan District
Seoul/Seongdong District
Seoul/Gwangjin District
Seoul/Dongdaemun District
Seoul/Jungnang District
Seoul/Seongbuk District
Seoul/Gangbuk District
Seoul/Dobong District
Seoul/Eunpyeong District
Seoul/Seod

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
print(hrm_seoul_venues.shape)
hrm_seoul_venues.head()

(790, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dartmouth(Morris Lake / Cole Harbour),44.662519,-63.483692,Bus stop 6471,44.661966,-63.483025,Bus Station
1,Dartmouth(Morris Lake / Cole Harbour),44.662519,-63.483692,Bissett Lake,44.661444,-63.482238,Lake
2,Dartmouth(Morris Lake / Cole Harbour),44.662519,-63.483692,Expert Crack & Concrete Repairs,44.661591,-63.486736,Construction & Landscaping
3,DartmouthEast Central(Portland Estates / South...,44.678888,-63.510144,Mike Mader Electrical Contracting,44.682977,-63.508228,Construction & Landscaping
4,DartmouthEast Central(Portland Estates / South...,44.678888,-63.510144,Penhorn Lake,44.674840,-63.512373,Beach


In [22]:
hrm_seoul_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BedfordNorthwest,1,1,1,1,1,1
BedfordSoutheast,6,6,6,6,6,6
Dartmouth(Morris Lake / Cole Harbour),3,3,3,3,3,3
DartmouthEast Central(Portland Estates / South Woodside / Woodlawn),2,2,2,2,2,2
DartmouthNorth,4,4,4,4,4,4
DartmouthNorth Central,4,4,4,4,4,4
DartmouthSouth Central(North Woodside),3,3,3,3,3,3
Eastern Passage,7,7,7,7,7,7
Gyeonggi/Ansan,3,3,3,3,3,3


In [23]:
print('There are {} uniques categories.'.format(len(hrm_seoul_venues['Venue Category'].unique())))

There are 178 uniques categories.


## Analyze Each Neighborhood

In [24]:
# one hot encoding
hrm_seoul_onehot = pd.get_dummies(hrm_seoul_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hrm_seoul_onehot['Neighborhood'] = hrm_seoul_venues['Neighborhood'] 

n = hrm_seoul_onehot.columns.get_loc('Neighborhood')

fixed_columns = [hrm_seoul_onehot.columns[n]] + list(hrm_seoul_onehot.columns[:n]) + list(hrm_seoul_onehot.columns[n+1:])
hrm_seoul_onehot = hrm_seoul_onehot[fixed_columns]

hrm_seoul_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bath House,Beach,Bed & Breakfast,Beer Bar,Big Box Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Bunsik Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Football Stadium,Forest,Fountain,Fried Chicken Joint,Gaming Cafe,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean BBQ Restaurant,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Noodle House,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outlet Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Sake Bar,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Track,Trail,Tunnel,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Wine Bar,Yoga Studio
0,Dartmouth(Morris Lake / Cole Harbour),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dartmouth(Morris Lake / Cole Harbour),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dartmouth(Morris Lake / Cole Harbour),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,DartmouthEast Central(Portland Estates / South...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
hrm_seoul_onehot.shape

(790, 179)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [26]:
hrm_seoul_grouped = hrm_seoul_onehot.groupby('Neighborhood').mean().reset_index()
hrm_seoul_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bath House,Beach,Bed & Breakfast,Beer Bar,Big Box Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Bunsik Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Football Stadium,Forest,Fountain,Fried Chicken Joint,Gaming Cafe,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean BBQ Restaurant,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Noodle House,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outlet Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Sake Bar,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Track,Trail,Tunnel,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Wine Bar,Yoga Studio
0,BedfordNorthwest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BedfordSoutheast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.166667,0.0,0.0,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Dart

In [27]:
hrm_seoul_grouped.shape

(64, 179)

Let's print each neighborhood along with the top 5 most common venues.

In [28]:
num_top_venues = 5

for hood in hrm_seoul_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = hrm_seoul_grouped[hrm_seoul_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BedfordNorthwest----
                  venue  freq
0                  Farm   1.0
1   American Restaurant   0.0
2   Rental Car Location   0.0
3          Noodle House   0.0
4  Other Great Outdoors   0.0


----BedfordSoutheast----
              venue  freq
0    Cosmetics Shop  0.17
1        Bagel Shop  0.17
2            Bakery  0.17
3  Basketball Court  0.17
4   Harbor / Marina  0.17


----Dartmouth(Morris Lake / Cole Harbour)----
                        venue  freq
0  Construction & Landscaping  0.33
1                        Lake  0.33
2                 Bus Station  0.33
3         American Restaurant  0.00
4                    Pharmacy  0.00


----DartmouthEast Central(Portland Estates / South Woodside / Woodlawn)----
                        venue  freq
0  Construction & Landscaping   0.5
1                       Beach   0.5
2         American Restaurant   0.0
3                    Pharmacy   0.0
4        Other Great Outdoors   0.0


----DartmouthNorth----
                 venue  freq


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hrm_seoul_grouped['Neighborhood']

for ind in np.arange(hrm_seoul_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hrm_seoul_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BedfordNorthwest,Farm,Yoga Studio,Cosmetics Shop,Forest,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,BedfordSoutheast,Cosmetics Shop,Bagel Shop,Middle Eastern Restaurant,Harbor / Marina,Bakery,Basketball Court,Distillery,Falafel Restaurant,Football Stadium,Food Truck
2,Dartmouth(Morris Lake / Cole Harbour),Construction & Landscaping,Lake,Bus Station,Yoga Studio,Event Space,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market
3,DartmouthEast Central(Portland Estates / South...,Construction & Landscaping,Beach,Yoga Studio,Event Space,Forest,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market
4,DartmouthNorth,Convenience Store,Bed & Breakfast,Gas Station,Insurance Office,Asian Restaurant,Falafel Restaurant,Fountain,Forest,Football Stadium,Food Truck


## Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

hrm_seoul_grouped_clustering = hrm_seoul_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hrm_seoul_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 4, 4, 3, 4, 4, 3, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hrm_seoul_merged = hrm_seoul

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
hrm_seoul_merged = hrm_seoul_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
hrm_seoul_merged = hrm_seoul_merged.dropna()
hrm_seoul_merged.head() # check the last columns!

,Region,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,HRM,Dartmouth(Morris Lake / Cole Harbour),44.662519,-63.483692,4.0,Construction & Landscaping,Lake,Bus Station,Yoga Studio,Event Space,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market
2,HRM,DartmouthEast Central(Portland Estates / South...,44.678888,-63.510144,4.0,Construction & Landscaping,Beach,Yoga Studio,Event Space,Forest,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market
3,HRM,DartmouthNorth Central,44.697761,-63.541843,3.0,Karaoke Bar,Sandwich Place,Coffee Shop,Pet Store,Yoga Studio,Dumpling Restaurant,Food Court,Fast Food Restaurant,Farmers Market,Farm
4,HRM,DartmouthSouth Central(North Woodside),44.667825,-63.552783,4.0,Donut Shop,Pizza Place,Lake,Electronics Store,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm
6,HRM,DartmouthNorth,44.678562,-63.579393,4.0,Convenience Store,Bed & Breakfast,Gas Station,Insurance Office,Asian Restaurant,Falafel Restaurant,Fountain,Forest,Football Stadium,Food Truck


Finally, visualize the resulting clusters.

In [33]:
hrm = hrm_seoul_merged[hrm_seoul_merged['Region'] == 'HRM'].reset_index(drop=True)
seoul = hrm_seoul_merged[hrm_seoul_merged['Region'] == 'Seoul'].reset_index(drop=True)

In [34]:
def getMapClusters(address, dataset, zoom_start=10, kclusters=kclusters):
    geolocator = Nominatim(user_agent=f'{address}_explorer')
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print(f'The geograpical coordinate of {address} are {latitude}, {longitude}.')
    
    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
    
    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(dataset['Latitude'], dataset['Longitude'], dataset['Neighborhood'], dataset['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

In [35]:
getMapClusters('Halifax', hrm, 9)

The geograpical coordinate of Halifax are 44.648618, -63.5859487.


![](image/hrm_cluster.png)

In [36]:
getMapClusters('Seoul', seoul, 9)

The geograpical coordinate of Seoul are 37.5666791, 126.9782914.


![](image/seoul_cluster.png)

In [37]:
for i in range(kclusters):
    print(f'Cluster Label: {i}')
    display(hrm_seoul_merged[hrm_seoul_merged['Cluster Labels'] == i])

Cluster Label: 0


,Region,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,HRM,BedfordNorthwest,44.734087,-63.752552,0.0,Farm,Yoga Studio,Cosmetics Shop,Forest,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Falafel Restaurant


Cluster Label: 1


,Region,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Seoul,Seoul/Dongdaemun District,37.579132,127.057221,1.0,Ice Cream Shop,Diner,Korean Restaurant,Bakery,Yoga Studio,Electronics Store,Football Stadium,Food Truck,Food Court,Fast Food Restaurant
35,Seoul,Seoul/Gangbuk District,37.640710,127.013272,1.0,Bath House,Ice Cream Shop,Korean Restaurant,Café,Yoga Studio,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market
38,Seoul,Seoul/Seodaemun District,37.574997,126.941155,1.0,Asian Restaurant,Coffee Shop,Korean Restaurant,Trail,Science Museum,Yoga Studio,Dumpling Restaurant,Food Truck,Food Court,Fast Food Restaurant
43,Seoul,Seoul/Geumcheon District,37.457775,126.902894,1.0,Chinese Restaurant,Ice Cream Shop,Korean Restaurant,Event Space,Dumpling Restaurant,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm
44,Seoul,Seoul/Yeongdeungpo District,37.519739,126.912303,1.0,Korean Restaurant,Chinese Restaurant,Asian Restaurant,Seafood Restaurant,Hotel,Diner,Park,BBQ Joint,Market,Coffee Shop
45,Seoul,Seoul/Dongjak District,37.496075,126.953734,1.0,Korean Restaurant,Coffee Shop,Bakery,Ice Cream Shop,Gym / Fitness Center,Pizza Place,College Cafeteria,Dumpling Restaurant,Chinese Restaurant,Fast Food Restaurant
50,Seoul,Seoul/Gangdong District,37.547522,127.149107,1.0,Café,Korean Restaurant,Bubble Tea Shop,Bus Stop,Bakery,Falafel Restaurant,Forest,Football Stadium,Food Truck,Food Court
51,Seoul,Incheon/Incheon,37.455791,126.705401,1.0,Korean Restaurant,Fast Food Restaurant,Coffee Shop,Yoga Studio,Electronics Store,Football Stadium,Food Truck,Food Court,Farmers Market,Farm
56,Seoul,Gyeonggi/Yongin,37.240941,127.177933,1.0,Korean Restaurant,Light Rail Station,Fast Food Restaurant,Dumpling Restaurant,Football Stadium,Food Truck,Food Court,Farmers Market,Farm,Falafel Restaurant
60,Seoul,Gyeonggi/Siheung,37.380050,126.802881,1.0,Korean Restaurant,Trail,Seafood Restaurant,Metro Station,Bus Stop,Bakery,Yoga Studio,Food Truck,Food Court,Fast Food Restaurant


Cluster Label: 2


,Region,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,HRM,HalifaxBedford Basin,44.672391,-63.657803,2.0,Playground,Park,Yoga Studio,Electronics Store,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
15,HRM,HalifaxSouth Central,44.643454,-63.631977,2.0,Trail,Park,Electronics Store,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
27,Seoul,Seoul/Jongno District,37.592128,126.979420,2.0,Scenic Lookout,Tourist Information Center,Historic Site,Tunnel,Trail,Food Court,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
62,Seoul,Gyeonggi/Euiwang,37.344718,126.968299,2.0,Trail,Noodle House,Electronics Store,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


Cluster Label: 3


,Region,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,HRM,DartmouthNorth Central,44.697761,-63.541843,3.0,Karaoke Bar,Sandwich Place,Coffee Shop,Pet Store,Yoga Studio,Dumpling Restaurant,Food Court,Fast Food Restaurant,Farmers Market,Farm
10,HRM,HalifaxLower Harbour,44.637785,-63.586997,3.0,Coffee Shop,Pizza Place,Theater,Spa,Sandwich Place,Bus Station,Bank,Restaurant,Breakfast Spot,Pharmacy
11,HRM,HalifaxMid-Harbour\n\nNova Scotia Provincial G...,44.645928,-63.575099,3.0,Coffee Shop,Restaurant,Seafood Restaurant,Hotel,Steakhouse,Gastropub,Bar,Sushi Restaurant,Historic Site,Pub
12,HRM,HalifaxUpper Harbour\n\nCanadian Forces(MARLANT),44.660803,-63.599256,3.0,Pizza Place,Coffee Shop,Sandwich Place,Breakfast Spot,Park,Chinese Restaurant,Sushi Restaurant,Steakhouse,Brewery,Deli / Bodega
13,HRM,HalifaxCentral,44.649777,-63.612403,3.0,Coffee Shop,Fast Food Restaurant,Restaurant,Sporting Goods Shop,Shopping Mall,Electronics Store,Clothing Store,Lingerie Store,Sandwich Place,Lebanese Restaurant
18,HRM,HalifaxWest(Bayers Lake / Clayton Park),44.656609,-63.669667,3.0,American Restaurant,Hotel,Restaurant,Breakfast Spot,Pizza Place,Coffee Shop,Electronics Store,Food Truck,Food Court,Fast Food Restaurant
22,HRM,BedfordSoutheast,44.726004,-63.668742,3.0,Cosmetics Shop,Bagel Shop,Middle Eastern Restaurant,Harbor / Marina,Bakery,Basketball Court,Distillery,Falafel Restaurant,Football Stadium,Food Truck
24,HRM,Lower SackvilleSouth,44.768456,-63.676567,3.0,Fast Food Restaurant,Playground,Athletics & Sports,Bus Line,Massage Studio,Scenic Lookout,Gym / Fitness Center,Park,Thrift / Vintage Store,Yoga Studio
28,Seoul,Seoul/Jung District,37.557335,126.997985,3.0,Coffee Shop,Korean Restaurant,Café,Park,Bakery,Theater,Athletics & Sports,Plaza,Chinese Restaurant,Noodle House
29,Seoul,Seoul/Yongsan District,37.528582,126.981987,3.0,Sandwich Place,Golf Course,Football Stadium,Bar,Movie Theater,Buffet,Steakhouse,Fried Chicken Joint,Taco Place,Thai Restaurant


Cluster Label: 4


,Region,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,HRM,Dartmouth(Morris Lake / Cole Harbour),44.662519,-63.483692,4.0,Construction & Landscaping,Lake,Bus Station,Yoga Studio,Event Space,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market
2,HRM,DartmouthEast Central(Portland Estates / South...,44.678888,-63.510144,4.0,Construction & Landscaping,Beach,Yoga Studio,Event Space,Forest,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market
4,HRM,DartmouthSouth Central(North Woodside),44.667825,-63.552783,4.0,Donut Shop,Pizza Place,Lake,Electronics Store,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm
6,HRM,DartmouthNorth,44.678562,-63.579393,4.0,Convenience Store,Bed & Breakfast,Gas Station,Insurance Office,Asian Restaurant,Falafel Restaurant,Fountain,Forest,Football Stadium,Food Truck
9,HRM,Eastern Passage,44.614385,-63.468725,4.0,Bus Station,Surf Spot,Playground,Soccer Field,Gaming Cafe,Bus Stop,Event Space,Food Court,Fast Food Restaurant,Farmers Market
17,HRM,HalifaxSouth,44.602119,-63.614009,4.0,Convenience Store,Pizza Place,High School,Dumpling Restaurant,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
25,HRM,Lower SackvilleWest,44.789793,-63.701987,4.0,National Park,Hobby Shop,Yoga Studio,Electronics Store,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
26,HRM,Lower SackvilleNorth,44.826875,-63.680785,4.0,Construction & Landscaping,Yoga Studio,Electronics Store,Football Stadium,Food Truck,Food Court,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
71,Seoul,Gyeonggi/Yangju,37.784028,127.045545,4.0,Chinese Restaurant,Asian Restaurant,Bus Station,Bar,Event Space,Forest,Football Stadium,Food Truck,Food Court,Fast Food Restaurant
